In [ ]:
#! pip install transformers datasets evaluate

In [ ]:
from datasets import load_dataset

In [ ]:
import pandas as pd

In [ ]:
from datasets import load_dataset
dataset_train = load_dataset("csv", data_files="archive/amazon Food Reviews 100k Dataset.csv", 
                       split="train[:10%]")

dataset_eval = load_dataset("csv", data_files="archive/amazon Food Reviews 100k Dataset.csv", 
                       split="train[10%:20%]")

In [ ]:
dataset_train = dataset_train.rename_column("Review", "text")\
    .rename_column("Rating", "label")\
    .remove_columns(['Id'])

dataset_eval = dataset_eval.rename_column("Review", "text")\
    .rename_column("Rating", "label")\
    .remove_columns(['Id'])

## Tokenizer

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
# apply tokenizer
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_dataset_train = dataset_train.map(tokenize_function, batched=True)
tokenized_dataset_eval = dataset_eval.map(tokenize_function, batched=True)

In [ ]:
# https://download.pytorch.org/whl/cu116
# !pip3 install torch torchvision torchaudio --extra-index-url 

## Process labels

In [ ]:
def minus_1(example):
    example["label"] = example["label"] - 1
    return example

In [ ]:
tokenized_dataset_train = tokenized_dataset_train.map(minus_1)
tokenized_dataset_eval = tokenized_dataset_eval.map(minus_1)

In [ ]:
tokenized_dataset_train.to_pandas().head()

## Load the model

Start by loading your model and specify the number of expected labels.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

In [ ]:
#model = model.to('cuda')

Training hyperparameters - Specify where to save the checkpoints from your training.
specify the evaluation_strategy parameter in your training arguments to report the evaluation metric at the end of each epoch:

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", 
                                  evaluation_strategy="epoch"
                                 )

## Evalute

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## Trainer

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_eval,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()